In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.io import loadmat, savemat
from scipy.signal import ellip, lfilter, filtfilt, find_peaks, butter, sosfiltfilt, sosfilt
from scipy.interpolate import interp1d

In [14]:
LB_pred_s3 = np.load('/Users/carlosaguila/PycharmProjects/BE521_Proj/pycharm - be521/LB_pred_s3.npy')
LB_pred_s2 = np.load('/Users/carlosaguila/PycharmProjects/BE521_Proj/pycharm - be521/LB_pred_s2.npy')
LB_pred_s1 = np.load('/Users/carlosaguila/PycharmProjects/BE521_Proj/pycharm - be521/LB_pred_s1.npy')

In [15]:
print(np.shape(LB_pred_s3))

(2949, 5)


In [16]:
def repeat_preds(preds, window_to_time_ratio=50):
    pred_all = []
    for row in preds:
        for i in range(window_to_time_ratio):
            pred_all.append(row)

    # For out problem, it is short 50 entries, so add the last row 50 more times
    for i in range(window_to_time_ratio):
        pred_all.append(row)
    
    return np.array(pred_all)

def interp_preds(preds, time_length):
    # N samples
    preds_sample_orig = np.arange(len(preds))
    
    # T time points
    preds_sample_target = np.arange(time_length)
    
    preds = preds.transpose()

    preds_interp = []
    
    for finger_preds in preds:
        f = interp1d(preds_sample_orig, finger_preds)
        new_preds = f(preds_sample_target)
        preds_interp.append(new_preds)
    
    preds_interp = np.array(preds_interp).transpose()
    
    return preds_interp

def compute_corr(preds, truth):
    subj_corr = []
    for i in range(5):
        finger_pred = preds.transpose()[i]
        finger_truth = truth.transpose()[i]
        subj_corr.append(pearsonr(finger_pred, finger_truth)[0])
    
    return subj_corr

In [30]:
LB_pred_s1_expanded = repeat_preds(LB_pred_s1,50)
LB_pred_s2_expanded = repeat_preds(LB_pred_s2,50)
LB_pred_s3_expanded = repeat_preds(LB_pred_s3,50)

In [33]:
print(np.shape(LB_pred_s3_expanded))

(147500, 5)


In [50]:
predictions = np.zeros((3,1),dtype=object)
predictions[0,0] = LB_pred_s1_expanded
predictions[1,0] = LB_pred_s2_expanded
predictions[2,0] = LB_pred_s3_expanded

savemat('/Users/carlosaguila/Downloads/predictions.mat', {'predicted_dg':predictions})

In [58]:
print(np.shape(predictions[2,0]))

(147500, 5)
